# Make Figure 6: Mitotic Cell Cycle

This notebooks takes all the trans genes that are significant in multiple cancers and runs a GSEA using Reactome. It then takes a subset of genes from the Mitotic Cell Cycle pathway and creates a heatmap. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.8.5


# Step 1: Run GSEA

First read in sig_pval_heatmap.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. 

In [3]:
sig_df = pd.read_csv('../Make_Tables/csv/mult_sig_pval_heatmap.csv') 

In [4]:
sig_df

,Proteomics,P_Value,Medians,Cancer
0,ARMH3,5.766739e-07,-0.405134,GBM
1,CUTC,8.514758e-07,-0.553255,GBM
2,PIP4K2A,2.783477e-06,-0.838882,GBM
3,DCTN1,1.500890e-05,-0.382988,GBM
4,ACTR1A,3.847715e-05,-0.491126,GBM
...,...,...,...,...
2685,GYG2,NaN,NaN,CO
2686,NDC80,NaN,0.345500,CO
2687,PIK3CD,NaN,-0.076000,CO
2688,TNFAIP8L2,NaN,-0.072400,CO


In [5]:
prot_list = list(sig_df.Proteomics.unique()) # list of genes with a sig pval in mult cancers
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', outdir='/Enrichr')

In [6]:
prot_enr.res2d.head(5)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,78/1631,1.086596e-16,1.662491e-13,0,0,2.748472,101.029209,DDX47;WDR3;WDR4;HNRNPU;NAT10;ZC3H8;ADAR;PPP2R2...,Reactome_2016
1,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",39/462,3.268842e-16,2.500664e-13,0,0,4.851470,172.988519,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;PPP2R...,Reactome_2016
2,Processing of Capped Intron-Containing Pre-mRN...,25/193,5.415573e-15,2.761942e-12,0,0,7.444464,244.546913,DDX5;DHX9;HNRNPU;USP39;ELAVL1;EFTUD2;NUP85;POL...,Reactome_2016
3,Cell Cycle Homo sapiens R-HSA-1640170,40/566,5.170850e-14,1.977850e-11,0,0,4.061573,124.256343,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;PPP2R...,Reactome_2016
4,rRNA modification in the nucleus Homo sapiens ...,14/58,9.127898e-13,2.793137e-10,0,0,13.872374,384.573712,UTP6;DDX47;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;P...,Reactome_2016


# Step 2: Get the list of significant genes 

In [19]:
index = 1
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))
genes

Cell Cycle, Mitotic Homo sapiens R-HSA-69278
total genes: 39


['TOP2A',
 'AHCTF1',
 'PCNA',
 'MCM7',
 'DCTN1',
 'NCAPG',
 'BUB1B',
 'PPP2R2A',
 'NCAPH',
 'ORC5',
 'ACTR1A',
 'PCM1',
 'XPO1',
 'NUP85',
 'NUF2',
 'TPR',
 'RAE1',
 'RAB8A',
 'RANBP2',
 'GINS2',
 'RFC3',
 'RFC4',
 'NUP210',
 'NUP155',
 'NUP133',
 'PRKCB',
 'RFC2',
 'GINS4',
 'CDK11B',
 'NDC80',
 'TPX2',
 'CENPF',
 'PPP2R2D',
 'MCM3',
 'MCM4',
 'MCM5',
 'OPTN',
 'MAD2L1',
 'MCM2']

In [20]:
# Catagorize genes
narrowed_cell_cycle = ['DCTN1', 'ACTR1A', 'MAD2L1', 'MCM4', 'PCNA', 'MCM5', 'MCM2','PCM1',
              'MCM7', 'TPR', 'MCM3', 'MCM6', 'NUP153', 'RFC3', 'CDK11B', 'XPO1','PRKCB', 'RFC2', 'NUF2', 
              'RAB8A', 'RFC4', 'RANBP2', 'CENPF', 'GINS4', 'NDC80', 'OPTN', 'PPP2R2A', 'PPP2R2D',
              'TPX2', 'GINS2', 'BUB1B', 'TOP2A', 'TOPBP1', 'NUP210', 'MSH2', 'MSH6', 'CDK11B', '']

dna_synth = ['MCM4', 'PCNA', 'MCM5', 'MCM2','PCM1', 'MCM7', 'MCM3', 'RFC3', 
             'RFC2', 'RFC4', 'GINS4', 'GINS2', 'TOP2A']

dynactin = ['DCTN1','ACTR1A']
spindle = ['BUB1B', 'CENPF', 'MAD2L1', 'NDC80', 'NUF2', 'PCM1', 'TPX2','DCTN1','ACTR1A']

checkpoints = ['BUB1B','PPP2R2A', 'PPP2R2D', 'TOPBP1','MAD2L1']

nuclear_pore_transport = ['TPR','NUP153','NUP210', 'XPO1','RANBP2']
other_transport = ['RAB8A']
other = ['PRKCB','OPTN','CDK11B']

path_list = {'DNA_Synth':dna_synth, 'Spindle':spindle, 'Dynactin': dynactin, 'Checkpoints':checkpoints,
            'Nuclear_Pore_Transport':nuclear_pore_transport}


In [21]:
bool_df = sig_df.Proteomics.isin(narrowed_cell_cycle)
df = sig_df[bool_df]
plot_df = df.loc[df['P_Value'] <= 0.05] # Only plot sig genes
plot_df

,Proteomics,P_Value,Medians,Cancer
3,DCTN1,0.000015,-0.382988,GBM
4,ACTR1A,0.000038,-0.491126,GBM
21,MCM4,0.001303,1.236484,GBM
24,MAD2L1,0.001303,0.546914,GBM
27,PCNA,0.001545,0.631486,GBM
...,...,...,...,...
2129,BUB1B,0.026111,-0.559000,EC
2143,NUF2,0.034789,-0.612500,EC
2148,CENPF,0.037588,-0.439000,EC
2157,TPR,0.042634,-0.184300,EC


In [22]:
ordered_prot_list = ['MSH2', 'MSH6', 'GINS2', 'GINS4', 'MCM2', 'MCM3', 'MCM4', 'MCM5', 'MCM6', 'MCM7',
                     'RFC2', 'RFC3', 'RFC4', 'RFC5',  'PCNA', 'TOP2A', 'CDK11B', 'PPP2R2A', 'PPP2R2D', 
                     'TOPBP1', 'MAD2L1', 'BUB1B', 'CENPF', 'NDC80', 'NUF2', 'TPX2', 'PCM1', 'DCTN1','ACTR1A']

In [23]:
# Reorder index to reorder the protein order in heatmap
plot_df["Index"] = plot_df["Proteomics"] + "_" + plot_df["Cancer"]
plot_df = plot_df.set_index("Index")

ordered_list = []
cancer_list = ['BR', 'CO', 'EC', 'GBM', 'HNSCC', 'LSCC', 'LUAD', 'OV']
for gene in ordered_prot_list:
    for cancer in cancer_list:
        ordered_list.append(gene+'_'+cancer)

ordered_df = plot_df.reindex(ordered_list)
ordered_df = ordered_df.loc[ordered_df['P_Value'] < 0.05] # Keep sig 

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
print('min p-value:', ordered_df.P_Value.min())

min p-value: 1.5008898009485894e-05


In [25]:
p.plotCircleHeatMap(ordered_df, circle_var='P_Value', color_var='Medians', x_axis='Proteomics', y_axis='Cancer',
                    plot_height=350, plot_width=800, x_axis_lab='', font_size=12, legend_max=0.05, 
                    legend_med = 0.001, legend_min=0.00001, save_png='Fig_6_Cell_Cycle.png')

Loading BokehJS ...

Figure(id='2898', ...)